In [139]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pprint

from bs4 import BeautifulSoup

# Start the driver

In [140]:
driver = webdriver.Firefox(executable_path='./geckodriver')
driver.implicitly_wait(5)
#ClimaTempo unfortunately takes a really long time to fully load, so we put a small timeout and catch the tiemoutException
driver.set_page_load_timeout(1)

# Load the first entry  point and acquire the states and cities data

In [ ]:
try:
    #first entry point provided
    driver.get("https://www.climatempo.com.br/climatologia/2/santos-sp") 
except TimeoutException:
    pass

def read_states(page):
    # Wait to load the dropdown list
    WebDriverWait(driver, 20).until(
        EC.invisibility_of_element_located((By.XPATH, "//select[@id='sel-state-geo']/option[@value='']")))
    WebDriverWait(driver, 5).until(
        EC.invisibility_of_element_located((By.XPATH, "//select[@id='sel-state-geo']/option[@value='0']")))
    
    states_element = page.find_element_by_id("sel-state-geo")   
    states = [x for x in states_element.find_elements_by_tag_name("option")]
    return states



def read_cities(states):
    cities = []
    for state in states:
        if len(cities) >= 100:
            print("Getting at least the only first 100 to faster processing...")
            return cities
        print("State loaded: {}({})".format(state.text, state.get_attribute("value")))
        state.click()
        
        # Wait to load the dropdown list
        WebDriverWait(driver, 5).until(
            EC.invisibility_of_element_located((By.XPATH, "//select[@id='sel-city-geo']/option[@value='0']")))
        
        cities_element_selector = driver.find_element_by_id("sel-city-geo")
        cities_elements = [x for x in cities_element_selector.find_elements_by_tag_name("option")]
        
        for city in cities_elements:
            cities.append((
                city.get_attribute("value")
                ,{ 
                "State": state.text,
                "City": city.text,
            }))
        
        
    return cities

elem = driver.find_element_by_xpath("/html/body/div[5]/div/div[2]/div[2]/div/div[2]/div[1]/div[1]/p[1]/span[2]")
elem.click()

states = read_states(driver)
cities = read_cities(states)


# Scrap into pages to acquire the data

In [138]:
def read_city_climatology(code, data):
    print("Scrapping data for {City}({State})".format(**data))
    try:
        #first entry point provided
        driver.get("https://www.climatempo.com.br/climatologia/{}/dummy".format(code)) 
    except TimeoutException:
        pass
        
    table_xpath = "/html/body/div[5]/div/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[3]/table/tbody"
    
    table_soup = BeautifulSoup(driver.find_element_by_xpath(table_xpath).get_attribute('outerHTML'))
    
    
    lines = table_soup.findChildren('tr')
    
    data['Climatology'] = {}
    
    for columns in lines:
        month = ''
        min_temp = 0
        max_temp = 0
        precipitation = 0
        for counter, column in enumerate(columns.findChildren('td')):

            
            if counter == 0:
                month = column.string
            elif counter == 1:
                min_temp = int(column.string.replace(u'\N{DEGREE SIGN}', ''))
            elif counter == 2:
                max_temp = int(column.string.replace(u'\N{DEGREE SIGN}', ''))
            elif counter == 3:
                precipitation = int(column.string)
                
        data['Climatology'][month] = {
            'Min': min_temp,
            'Max': max_temp,
            'Precipitation': precipitation
        }
    
       
read_city_climatology(cities[0][0], cities[0][1])
read_city_climatology(cities[1][0], cities[1][1])

        
#for code, data in enumcities:
#
#    read_city_climatology(code, data)

Scrapping data for Rio Branco(Acre)


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=57851): Max retries exceeded with url: /session/331e3a8b-11f0-41c9-94b5-c7da151a815b/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fe689f73a58>: Failed to establish a new connection: [Errno 111] Connection refused',))

# Print scrapped data

In [ ]:
for _, data in cities:
    pprint.pprint(data)
